In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Keras Imports
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, CuDNNGRU, Dropout, Bidirectional, Conv1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence 
from keras.wrappers.scikit_learn import KerasClassifier
# Numpy
import numpy
numpy.random.seed(1331)
# Pandas
import pandas as pd
# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_val_score
# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
# Garbage Collector
import gc
import sys
from keras.optimizers import Adam

import time
# Any results you write to the current directory are saved as output.

In [ ]:
from tqdm import tqdm

# Train 1st Model -- GRU 

In [ ]:
start= time.clock()

q_df = pd.read_csv("../input/train.csv")
test = pd.read_csv('../input/test.csv')

X_train = q_df.question_text
y_train = q_df.target

x_test = test.question_text

from keras.preprocessing.text import Tokenizer

# create a tokenizer 
tot_uniq_words= 49681
token = Tokenizer(num_words=tot_uniq_words)
token.fit_on_texts(q_df['question_text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
maxlen = 134
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=maxlen)
test_seq_x = sequence.pad_sequences(token.texts_to_sequences(x_test), maxlen=maxlen)

In [ ]:
del [X_train, x_test]
gc.collect()

In [ ]:
batches= 3152
drpt_amt = .2
embed_size = 88
epochs =2
gru1_nrns = 47
gru2_nrns= 49

In [ ]:

K.clear_session()
model = Sequential()
model.add(Embedding(tot_uniq_words, embed_size, input_length=maxlen))
model.add(Dropout(drpt_amt))
model.add(Bidirectional(CuDNNGRU(gru1_nrns, return_sequences=True)))
model.add(Dropout(drpt_amt))
model.add(Bidirectional(CuDNNGRU(gru2_nrns)))
model.add(Dropout(drpt_amt))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit(x = train_seq_x, y = np.array(y_train), epochs= epochs, batch_size=batches)


gru_preds= model.predict(train_seq_x)
gru_x_test = model.predict(test_seq_x)


In [ ]:
del [train_seq_x, token, model]
gc.collect() 

# Train 2nd Model -- DPCNN

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.models import Model
from keras.layers import Input, Dense, Embedding, MaxPooling1D, Conv1D, SpatialDropout1D
from keras.layers import add, Dropout, PReLU, BatchNormalization, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import optimizers
from keras import initializers, regularizers, constraints, callbacks


from datetime import datetime


from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from keras.models import Model
from keras.layers import Input, Dense, Embedding, MaxPooling1D, Conv1D, SpatialDropout1D
from keras.layers import add, Dropout, PReLU, BatchNormalization, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import optimizers
from keras import initializers, regularizers, constraints, callbacks
# Any results you write to the current directory are saved as output.

import time

from keras.models import load_model
from sklearn.metrics import f1_score, roc_auc_score
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold

In [ ]:

# DPCNN

train_df = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

X_train = train_df.question_text
y_train = train_df.target

X_test = test.question_text

max_features= 100000
maxlen = 134
embed_size = 300

### GETTING 3% HOLDOUT



contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

X_train = X_train.apply(lambda x: clean_contractions(x, contraction_mapping))

X_test = X_test.apply(lambda x: clean_contractions(x, contraction_mapping))

# deal with punctuations

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'


punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

X_train = X_train.apply(lambda x: clean_special_chars(x, punct, punct_mapping))

X_test = X_test.apply(lambda x: clean_special_chars(x, punct, punct_mapping))


mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

X_train = X_train.apply(lambda x: correct_spelling(x, mispell_dict))

X_test = X_test.apply(lambda x: correct_spelling(x, mispell_dict))

In [ ]:
print('preprocessing start')

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train)) 
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

# word_index = tokenizer.word_index
nb_words = len(tokenizer.word_index)+1
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in tqdm(tokenizer.word_index.items()):
#     if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [ ]:
epochs = 2
batch_size = 500
spatial_dropout=0.11260266916085344
dense_nr = 64
filter_nr=64
filter_size=4
max_pool_size= 3
dense_dropout = 0.5365018989527698
max_pool_strides=2,
conv_kern_reg= 1e-06
conv_bias_reg=1e-05
nb_words=nb_words
embedding_matrix= embedding_matrix

In [ ]:

K.clear_session()
comment = Input(shape=(maxlen,))
emb_comment = Embedding(nb_words, embed_size, weights= [embedding_matrix], trainable= False)(comment)
emb_comment = SpatialDropout1D(spatial_dropout)(emb_comment)

block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(emb_comment)
block1 = BatchNormalization()(block1)
block1 = PReLU()(block1)
block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block1)
block1 = BatchNormalization()(block1)
block1 = PReLU()(block1)

#we pass embedded comment through conv1d with filter size 1 because it needs to have the same shape as block output
#if you choose filter_nr = embed_size (300 in this case) you don't have to do this part and can add emb_comment directly to block1_output
resize_emb = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(emb_comment)
resize_emb = PReLU()(resize_emb)

block1_output = add([block1, resize_emb])
block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output)

block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block1_output)
block2 = BatchNormalization()(block2)
block2 = PReLU()(block2)
block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block2)
block2 = BatchNormalization()(block2)
block2 = PReLU()(block2)

block2_output = add([block2, block1_output])
block2_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output)

block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block2_output)
block3 = BatchNormalization()(block3)
block3 = PReLU()(block3)
block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block3)
block3 = BatchNormalization()(block3)
block3 = PReLU()(block3)

block3_output = add([block3, block2_output])
block3_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output)

block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block3_output)
block4 = BatchNormalization()(block4)
block4 = PReLU()(block4)
block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block4)
block4 = BatchNormalization()(block4)
block4 = PReLU()(block4)

block4_output = add([block4, block3_output])
block4_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block4_output)

block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block4_output)
block5 = BatchNormalization()(block5)
block5 = PReLU()(block5)
block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block5)
block5 = BatchNormalization()(block5)
block5 = PReLU()(block5)

block5_output = add([block5, block4_output])
block5_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block5_output)

block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block5_output)
block6 = BatchNormalization()(block6)
block6 = PReLU()(block6)
block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block6)
block6 = BatchNormalization()(block6)
block6 = PReLU()(block6)

block6_output = add([block6, block5_output])
block6_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block6_output)

block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block6_output)
block7 = BatchNormalization()(block7)
block7 = PReLU()(block7)
block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=regularizers.l2(conv_kern_reg), bias_regularizer=regularizers.l2(conv_bias_reg))(block7)
block7 = BatchNormalization()(block7)
block7 = PReLU()(block7)

block7_output = add([block7, block6_output])
output = GlobalMaxPooling1D()(block7_output)

output = Dense(dense_nr, activation='linear')(output)
output = BatchNormalization()(output)
output = PReLU()(output)
output = Dropout(dense_dropout)(output)
output = Dense(1, activation='sigmoid')(output)

model = Model(comment, output)


model.compile(loss='binary_crossentropy', 
            optimizer=optimizers.Adam(),
            metrics=['accuracy'])
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x = x_train, y = np.array(y_train), epochs= epochs, batch_size=batch_size)

### NOTE
# this goes to level 1 
dpcnn_preds = model.predict(x_train, verbose=1)
# this goes to prediction
dpcnn_x_test = model.predict(x_test, verbose=1)


In [ ]:
del [x_train,  X_train, x_test, X_test, y_train, tokenizer, train_df, max_features,model,all_embs, q_df, test_seq_x, word_index]
del [embedding_vector, embeddings_index, embedding_matrix]
gc.collect()

In [ ]:
import gc

In [ ]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *


maxlen = 70 # max number of words in a question to use

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

## fill up the missing values
train_X = train_df["question_text"].fillna("_##_").values
test_X = test_df["question_text"].fillna("_##_").values

## Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
x_train = pad_sequences(train_X, maxlen=maxlen)
x_test = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
y_train = train_df['target'].values

In [ ]:
del[train_df, train_X ]
gc.collect()

In [ ]:
EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

# word_index = tokenizer.word_index
nb_words = len(tokenizer.word_index)+1
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in tqdm(tokenizer.word_index.items()):
#     if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

## some config values 
embed_size = 300 # how big is each word vector
max_features = nb_words # how many unique words to use (i.e num rows in embedding vector)


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
    

In [ ]:
# create model

K.clear_session()
    # Inputs

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
x = SpatialDropout1D(0.1)(x)
x = Bidirectional(CuDNNLSTM(40, return_sequences=True))(x)
y = Bidirectional(CuDNNGRU(40, return_sequences=True))(x)

atten_1 = Attention(maxlen)(x) # skip connect
atten_2 = Attention(maxlen)(y)
avg_pool = GlobalAveragePooling1D()(y)
max_pool = GlobalMaxPooling1D()(y)

conc = concatenate([atten_1, atten_2, avg_pool, max_pool])
conc = Dense(16, activation="relu")(conc)
conc = Dropout(0.1)(conc)
outp = Dense(1, activation="sigmoid")(conc)    

model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

clr = CyclicLR(base_lr=0.001, max_lr=0.002,
           step_size=300., mode='exp_range',
           gamma=0.99994)

model.fit(x_train, np.array(y_train), batch_size=512, epochs=2, callbacks = [clr], verbose=1)    

attn_preds = model.predict(x_train, verbose=1)
attn_x_test = model.predict(x_test, verbose=1)



In [ ]:
del model
gc.collect()

# Meta Learner

In [ ]:
target = np.array(y_train)

In [ ]:
level_1_metalearner = pd.DataFrame(columns = ['attn', 'dpcnn', 'gru','target'])

In [ ]:
level_1_metalearner['attn'] = attn_preds.ravel()
level_1_metalearner['dpcnn'] = dpcnn_preds.ravel()
level_1_metalearner['gru'] = gru_preds.ravel()
level_1_metalearner['target'] = target.ravel()

In [ ]:
level_1_metalearner.head()

# Level 1 table for test units


In [ ]:
level_1_test = pd.DataFrame(pd.DataFrame(columns = ['attn_test', 'dpcnn_test', 'gru_test']))

level_1_test.attn_test = attn_x_test.ravel()
level_1_test.dpcnn_test = dpcnn_x_test.ravel()
level_1_test.gru_test = gru_x_test.ravel()

In [ ]:
level_1_test.head()

# Train meta learner

In [ ]:
x_train = level_1_metalearner.drop('target', axis=1)

In [ ]:
y_train = level_1_metalearner.target

In [ ]:
act1= 'relu'
act2= 'tanh'
act3= 'tanh'
batch_size= 100000
dense_nr1= 287
dense_nr2= 670
dense_nr3= 140
dropout1= 0.3383299306103077
dropout2= 0.43369555270628113
dropout3= 0.543347139093403
epochs= 70

In [ ]:
# create model

K.clear_session()
model= Sequential()
model.add(Dense(dense_nr1, activation=act1, input_shape=(3,)))
model.add(Dropout(dropout1))
model.add(Dense(dense_nr2, activation=act2))
model.add(Dropout(dropout2))
model.add(Dense(dense_nr3, activation=act3))
model.add(Dropout(dropout3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
            optimizer='adam',
            metrics=['accuracy'])

model.fit(x_train, np.array(y_train), batch_size=batch_size, epochs=epochs, verbose=1)    

metalearner_x_test = model.predict(level_1_test, verbose=1)



In [ ]:
test_df.drop('question_text', axis=1, inplace=True)

In [ ]:
test_df['prediction'] = metalearner_x_test.ravel()

In [ ]:
test_df.prediction = test_df.prediction.apply(lambda x: 1 if x>=.34 else 0)

In [ ]:
test_df.head()

In [ ]:
print('process time in minutes: ', (time.clock()-start)/60)

In [ ]:
test_df.to_csv('submission.csv', index=False)